## Geographic distributions of four main clades in New York City

In [3]:
import os
import json
import warnings
import pandas as pd
import folium

warnings.filterwarnings('ignore')

In [4]:
meta = pd.read_csv('../data/rawdata/nyc_omicron_metadata.csv', index_col='seqName')

In [5]:
nextclade = pd.read_csv('../data/nextclade_nyc/nextclade.tsv', delimiter='\t', index_col='seqName')

In [6]:
clades = {
    'A': [],
    'B': [],
    'C': [],
    'D': [],
    'Others': []
}
for i in nextclade.index:
    substitutions = nextclade.loc[i, 'substitutions'].split(',')
    if 'G5515T' in substitutions:
        clades['A'].append(i)
    elif 'G5924A' in substitutions:
        clades['B'].append(i)
    elif 'T10135C' in substitutions and 'C25708T' in substitutions and 'A29301G' in substitutions:
        clades['C'].append(i)
    elif 'C2470T' in substitutions and 'G22599A' in substitutions:
        clades['D'].append(i)
    else:
        clades['Others'].append(i)

In [7]:
def convert_zipcode(zipcode):
    if '-' in zipcode:
        return zipcode.split('-')[0]
    else:
        return zipcode


nyc_zipcode = []
with open('../data/rawdata/MODZCTA_2010_WGS1984.geo.json') as f:
    nyc_json = json.load(f)

for i in nyc_json['features']:
    nyc_zipcode.append(i['properties']['MODZCTA'])

meta['Postal'] = meta['Postal'].apply(convert_zipcode)

In [8]:
colors = {
    'A': ('Blues', 0.6),
    'B': ('YlOrRd', 0.4),
    'C': ('Purples', 0.8),
    'D': ('OrRd', 1.0),
}

for c in clades:
    if c != 'Others':
        submeta = meta[meta.index.isin(clades[c])]
        df = submeta[submeta['Postal'].isin(nyc_zipcode)]['Postal'].value_counts().to_frame().reset_index().rename(columns={'index':'ZIP', 'Postal':'Count'})
        df['Count'] = 1
        m = folium.Map(location=[40.693943, -73.985880], default_zoom_start=12, tiles=None)

        m.choropleth(geo_data='../data/rawdata/MODZCTA_2010_WGS1984.geo.json',
                    data=df,
                    columns=['ZIP', 'Count'],
                    key_on='feature.properties.MODZCTA',
                    fill_color=colors[c][0],
                    fill_opacity=colors[c][1],
                    line_opacity=0.3,
                    nan_fill_color='white')

        m.save('../figures/Figure_1D_Clade'+c+'.html')